In [1]:
import pandas as pd
import re

## Inside Airbnb Data

In [2]:
air_df = pd.read_csv("../data/air-bnb-listings.csv", sep=';')
air_df['City'] = air_df['City'].str.replace('-', ' ')
# Convert to uppercase for later matching
air_df['City'] = air_df['City'].str.upper()
# Strip 'City' from City names (e.g. NEW YORK not NEW YORK CITY)
# But also converts JERSEY CITY to JERSEY
air_df['City'] = air_df['City'].str.replace(' CITY', '')
air_df['City'] = air_df['City'].str.replace(' COUNTY', '')
air_df.head()

,Room ID,Name,Host ID,Neighbourhood,Room type,Room Price,Minimum nights,Number of reviews,Date last review,Number of reviews per month,Rooms rent by the host,Availibility,Updated Date,City,Country,Coordinates,Location
0,25540031,Villa Giusè (rosa marina),192701298,Ostuni,Entire home/apt,140,8,2,2019-03-22,0.1,1,318,2020-02-28,PUGLIA,Italy,"40.7922335859,17.5537752385","Italy, Puglia, Ostuni"
1,25558372,CASALE RIFORMA,192812326,Carovigno,Entire home/apt,80,7,0,NaN,NaN,2,365,2020-02-28,PUGLIA,Italy,"40.6935039587,17.7587767393","Italy, Puglia, Carovigno"
2,25563657,Appartamento nel Salento,90976022,Parabita,Entire home/apt,80,1,0,NaN,NaN,2,365,2020-02-28,PUGLIA,Italy,"40.0533228907,18.1303956856","Italy, Puglia, Parabita"
3,25575222,Brezza Marina - stanza privata climatizzata,17154685,Pulsano,Private room,48,4,0,NaN,NaN,2,0,2020-02-28,PUGLIA,Italy,"40.3638944835,17.3491268808","Italy, Puglia, Pulsano"
4,25652335,camera kappa vista porto con bagno in camera,193183493,Brindisi,Private room,49,1,0,NaN,NaN,1,215,2020-02-28,PUGLIA,Italy,"40.6368142665,17.9478514701","Italy, Puglia, Brindisi"


#### Global

In [3]:
airbnb_city = pd.Series(air_df['City'].unique())
len(airbnb_city)

101

In [4]:
airbnb_countries = pd.Series(air_df['Country'].unique())
len(airbnb_countries)

30

#### United States

Notice that dataset still contains 'NV', 'OR, 'MSA', 'DC'

In [5]:
air_us_cities = air_df[air_df['Country'].str.contains('United states')]
airbnb_us_cities = pd.Series(sorted(air_us_cities['City'].unique()))
airbnb_us_cities

0           ASHEVILLE
1              AUSTIN
2              BOSTON
3             BROWARD
4           CAMBRIDGE
5             CHICAGO
6            CLARK NV
7            COLUMBUS
8              DENVER
9              HAWAII
10             JERSEY
11        LOS ANGELES
12          NASHVILLE
13        NEW ORLEANS
14           NEW YORK
15            OAKLAND
16      PACIFIC GROVE
17           PORTLAND
18       RHODE ISLAND
19           SALEM OR
20          SAN DIEGO
21      SAN FRANCISCO
22          SAN MATEO
23        SANTA CLARA
24         SANTA CRUZ
25            SEATTLE
26    TWIN CITIES MSA
27      WASHINGTON DC
dtype: object

## Kaggle Cost of Living, etc. for Global Cities

In [6]:
costs_df = pd.read_csv("../data/Cost_of_living_index.csv")
costs_df.rename(columns = {'City':'City_Country'}, inplace = True)
costs_df

,Rank,City_Country,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index
0,1,"Hamilton, Bermuda",137.56,103.03,121.21,126.56,151.77,114.19
1,2,"Zurich, Switzerland",128.65,62.62,97.39,127.35,127.14,142.39
2,3,"Basel, Switzerland",126.89,46.14,88.66,120.44,129.10,141.48
3,4,"Lausanne, Switzerland",119.62,50.35,86.83,116.35,122.83,132.58
4,5,"Bern, Switzerland",118.42,39.22,80.93,114.54,114.86,115.48
...,...,...,...,...,...,...,...,...
531,532,"Vadodara, India",22.28,3.78,13.52,22.26,14.43,74.59
532,533,"Kochi, India",21.73,5.45,14.02,21.93,12.20,73.42
533,534,"Visakhapatnam, India",21.52,4.26,13.35,21.09,17.09,55.47
534,535,"Navi Mumbai, India",19.30,4.70,12.39,19.21,12.55,138.06


In [7]:
# Split City and Country
cc_df = costs_df['City_Country'].apply(lambda x: pd.Series(x.split(',', 1)))

# Remove State Abbreviations from US entries
cc_df[1] = cc_df[1].apply(lambda x: pd.Series(re.sub(r'\w*,\s', '',x)))

# Clean up Dataframe
city_costs_df = pd.concat([cc_df, costs_df], axis=1)
city_costs_df.rename(columns={0:'City',1:'Country'},inplace=True)

# Convert to uppercase for later matching
city_costs_df['City'] = city_costs_df['City'].str.upper()

city_costs_df['City'] = city_costs_df['City'].str.replace(' CITY', '')
city_costs_df['City'] = city_costs_df['City'].str.replace(' COUNTY', '')

#### Number of Cities

In [8]:
city_costs = pd.Series(city_costs_df['City'].unique())
len(city_costs)

526

#### Number of Countries

In [9]:
country_costs = pd.Series(city_costs_df['Country'].unique())
len(country_costs)

115

## Kaggle Population for Global Cities

In [10]:
pop_df = pd.read_csv('../data/cities15000.csv', encoding = "ISO-8859-1")
# Convert to uppercase for later matching
pop_df['asciiname'] = pop_df['asciiname'].str.upper()

pop_df['asciiname'] = pop_df['asciiname'].str.replace(' CITY', '')
pop_df['asciiname'] = pop_df['asciiname'].str.replace(' COUNTY', '')

pop_df.head()

,geonameid,name,asciiname,alternatenames,latitude,longitude,feature class,feature code,country code,cc2,admin1 code,admin2 code,admin3 code,admin4 code,population,elevation,dem,timezone,modification date
0,3040051,les Escaldes,LES ESCALDES,"Ehskal'des-Ehndzhordani,Escaldes,Escaldes-Engo...",42.50729,1.53414,P,PPLA,AD,NaN,8,NaN,NaN,NaN,15853,NaN,1033.0,Europe/Andorra,10/15/08
1,3041563,Andorra la Vella,ANDORRA LA VELLA,"ALV,Ando-la-Vyey,Andora,Andora la Vela,Andora ...",42.50779,1.52109,P,PPLC,AD,NaN,7,NaN,NaN,NaN,20430,NaN,1037.0,Europe/Andorra,5/30/10
2,290594,Umm al Qaywayn,UMM AL QAYWAYN,"Oumm al Qaiwain,Oumm al QaÌøwaÌøn,Um al Kawain...",25.56473,55.55517,P,PPLA,AE,NaN,7,NaN,NaN,NaN,44411,NaN,2.0,Asia/Dubai,10/7/14
3,291074,Ras al-Khaimah,RAS AL-KHAIMAH,"Julfa,Khaimah,RKT,Ra's al Khaymah,Ra's al-Chai...",25.78953,55.9432,P,PPLA,AE,NaN,5,NaN,NaN,NaN,115949,NaN,2.0,Asia/Dubai,12/5/15
4,291696,Khawr Fakkn,KHAWR FAKKAN,"Fakkan,Fakkn,Khawr Fakkan,Khawr Fakkn,Khaw...",25.33132,56.34199,P,PPL,AE,NaN,6,NaN,NaN,NaN,33575,NaN,20.0,Asia/Dubai,10/25/13


#### Total Cities

In [11]:
city_pop = pd.Series(pop_df['asciiname'].unique())
len(city_pop)

22229

#### Total Country Codes

In [12]:
country_pop = pd.Series(pop_df['country code'].unique())
len(country_pop)

245

## Find Common Cities

### Cities in both population and Cost of X Sets

In [13]:
pop_costs = pd.Series(list(set(city_costs) & set(city_pop)))
pop_costs

0                 LYON
1                 BONN
2             ADELAIDE
3              CRAIOVA
4               PADOVA
            ...       
487    WEST PALM BEACH
488             SYDNEY
489         PORTSMOUTH
490          JERUSALEM
491            WINDSOR
Length: 492, dtype: object

### Cities in Cost Set but not Population - (Spelling differences, etc.)

In [14]:
cost_cities = pd.Series(list(set(city_costs) - set(city_pop)))
cost_cities

0     ZARAGOZA (SARAGOSSA)
1                AD DAMMAM
2                BANGALORE
3                QUERETARO
4              ULAANBAATAR
5        LUCKNOW (LAKHNAU)
6                    MALMO
7                AL KHOBAR
8               GOTHENBURG
9                   SLIEMA
10                  EXETER
11              DUSSELDORF
12                 SWINDON
13             SAINT LOUIS
14                 COLOGNE
15             BHUBANESWAR
16               EDINBURGH
17         KRAKOW (CRACOW)
18                  PENANG
19                 ANTWERP
20    THE HAGUE (DEN HAAG)
21                 PATTAYA
22                SELANGOR
23         JEDDAH (JIDDAH)
24               FRANKFURT
25               NUREMBERG
26                   IRBIL
27           TEL AVIV-YAFO
28       SEVILLE (SEVILLA)
29         NIZHNY NOVGOROD
30       PALMA DE MALLORCA
31                    KIEV
32                  GALWAY
33                 GLASGOW
dtype: object

### Cities in Airbnb Dataset and Cost of X Sets

In [15]:
air_cost_list = pd.Series(list(set(airbnb_city) & set(city_costs)))
len(air_cost_list)

67

### Cities and Airbnb, Population, AND Cost of X Sets

#### What cities can't we find a match for in the common dataset (of Pop and Cost of X)
* Edinburgh is spelled Edinburg
* Hawaii is a state, Barwon South West Vic - region/provinence
* Visualisations?

Gives us an idea of extent of data cleaning we would need to automate it


In [16]:
# Airbnb minus the intersection of found cities
missing_cities = (set(airbnb_city)) - (set(airbnb_city) & set(pop_costs))
missing_cities                        

{'ANTWERP',
 'BAROSSA VALLEY',
 'BARWON SOUTH WEST VIC',
 'BELIZE',
 'BERGAMO',
 'BROWARD',
 'CLARK NV',
 'CRETE',
 'EDINBURGH',
 'EUSKADI',
 'GHENT',
 'GIRONA',
 'GREATER MANCHESTER',
 'HAWAII',
 'MALLORCA',
 'MENORCA',
 'NEW BRUNSWICK',
 'NORTHERN RIVERS',
 'PACIFIC GROVE',
 'PUGLIA',
 'RHODE ISLAND',
 'SALEM OR',
 'SAN MATEO',
 'SANTA CLARA',
 'SANTA CRUZ',
 'SEVILLA',
 'SICILY',
 'SOUTH AEGEAN',
 'TASMANIA',
 'TRENTINO',
 'TWIN CITIES MSA',
 'VAUD',
 'VENICE',
 'VISUALISATIONS',
 'WASHINGTON DC',
 'WESTERN AUSTRALIA'}

#### Cities Found in Airbnb AND Population AND Cost of X

In [17]:
air_pop_cost_list = sorted((list(set(airbnb_city) & set(pop_costs))))
print(len(air_pop_cost_list))
print(air_pop_cost_list)

65
['AMSTERDAM', 'ASHEVILLE', 'ATHENS', 'AUSTIN', 'BARCELONA', 'BEIJING', 'BERLIN', 'BOLOGNA', 'BORDEAUX', 'BOSTON', 'BRISTOL', 'BRUSSELS', 'BUENOS AIRES', 'CAMBRIDGE', 'CAPE TOWN', 'CHICAGO', 'COLUMBUS', 'COPENHAGEN', 'DENVER', 'DUBLIN', 'FLORENCE', 'GENEVA', 'HONG KONG', 'ISTANBUL', 'JERSEY', 'LISBON', 'LONDON', 'LOS ANGELES', 'LYON', 'MADRID', 'MALAGA', 'MELBOURNE', 'MEXICO', 'MILAN', 'MONTREAL', 'MUNICH', 'NAPLES', 'NASHVILLE', 'NEW ORLEANS', 'NEW YORK', 'OAKLAND', 'OSLO', 'OTTAWA', 'PARIS', 'PORTLAND', 'PORTO', 'PRAGUE', 'QUEBEC', 'RIO DE JANEIRO', 'ROME', 'SAN DIEGO', 'SAN FRANCISCO', 'SANTIAGO', 'SEATTLE', 'SINGAPORE', 'STOCKHOLM', 'SYDNEY', 'TAIPEI', 'THESSALONIKI', 'TOKYO', 'TORONTO', 'VALENCIA', 'VANCOUVER', 'VICTORIA', 'VIENNA']


#### Total listings for All Cities in Airbnb, Population, and Cost of X Sets

In [18]:
global_cities = air_df.groupby("City", as_index=False)["Room ID"].count()
global_city_listings = global_cities[global_cities['City'].isin(air_pop_cost_list)==True]
pd.Series(global_city_listings['Room ID']).sum()

1051854

#### US Cities in Airbnb and Population and Cost of X Sets

In [19]:
us_air_pop_cost = pd.Series(list(set(airbnb_us_cities) & set(pop_costs))).tolist()
print(len(us_air_pop_cost))
print(us_air_pop_cost)

17
['NEW ORLEANS', 'OAKLAND', 'CAMBRIDGE', 'ASHEVILLE', 'NEW YORK', 'DENVER', 'AUSTIN', 'SEATTLE', 'CHICAGO', 'NASHVILLE', 'COLUMBUS', 'JERSEY', 'SAN FRANCISCO', 'SAN DIEGO', 'LOS ANGELES', 'PORTLAND', 'BOSTON']


#### Total Listings for US Cities in Airbnb, Population, and Cost of X Sets

In [20]:
cities = air_df.groupby("City", as_index=False)["Room ID"].count()
city_listings = cities[cities['City'].isin(us_air_pop_cost)==True]
pd.Series(city_listings['Room ID']).sum()

179834

#### City Explorations

In [21]:
amst_df = pd.read_csv("../data/Amsterdam_08June2020.csv")
amst_df

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2818,Quiet Garden View Room & Super Fast WiFi,3159,Daniel,NaN,Oostelijk Havengebied - Indische Buurt,52.36575,4.94142,Private room,59,3,278,2020-02-14,2.04,1,158
1,9693,Top Location on Canal (Center Flat),32366,Sabine And Sander,NaN,Centrum-West,52.37802,4.89270,Entire home/apt,119,3,55,2018-06-29,0.44,1,17
2,20168,Studio with private bathroom in the centre 1,59484,Alexander,NaN,Centrum-Oost,52.36509,4.89354,Private room,100,1,340,2020-04-09,2.72,2,31
3,25428,Lovely apt in City Centre (w.lift) near Jordaan,56142,Joan,NaN,Centrum-West,52.37297,4.88339,Entire home/apt,125,14,5,2020-02-09,0.17,1,212
4,27886,"Romantic, stylish B&B houseboat in canal district",97647,Flip,NaN,Centrum-West,52.38761,4.89188,Private room,135,2,217,2020-03-02,2.12,1,235
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19347,43707738,Spacious beautiful apt mins from the city center,170915281,Lara,NaN,Centrum-Oost,52.36881,4.92793,Entire home/apt,161,2,0,NaN,NaN,1,54
19348,43707910,Eco-friendly Triple Room in Westerpark,237371423,Conscious Hotel Westerpark,NaN,Westerpark,52.38553,4.87647,Private room,95,1,0,NaN,NaN,7,358
19349,43708450,Conscious WOW Room with a bathtub and a super ...,237371423,Conscious Hotel Westerpark,NaN,Westerpark,52.38571,4.87612,Private room,100,1,0,NaN,NaN,7,344
19350,43709001,Eco-friendly Big Double Room with a huge bed,237371423,Conscious Hotel Westerpark,NaN,Westerpark,52.38596,4.87441,Private room,75,1,0,NaN,NaN,7,359


In [22]:
amst_df.room_type.unique()

array(['Private room', 'Entire home/apt', 'Hotel room', 'Shared room'],
      dtype=object)

In [23]:
city_costs_df

,City,Country,Rank,City_Country,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index
0,HAMILTON,Bermuda,1,"Hamilton, Bermuda",137.56,103.03,121.21,126.56,151.77,114.19
1,ZURICH,Switzerland,2,"Zurich, Switzerland",128.65,62.62,97.39,127.35,127.14,142.39
2,BASEL,Switzerland,3,"Basel, Switzerland",126.89,46.14,88.66,120.44,129.10,141.48
3,LAUSANNE,Switzerland,4,"Lausanne, Switzerland",119.62,50.35,86.83,116.35,122.83,132.58
4,BERN,Switzerland,5,"Bern, Switzerland",118.42,39.22,80.93,114.54,114.86,115.48
...,...,...,...,...,...,...,...,...,...,...
531,VADODARA,India,532,"Vadodara, India",22.28,3.78,13.52,22.26,14.43,74.59
532,KOCHI,India,533,"Kochi, India",21.73,5.45,14.02,21.93,12.20,73.42
533,VISAKHAPATNAM,India,534,"Visakhapatnam, India",21.52,4.26,13.35,21.09,17.09,55.47
534,NAVI MUMBAI,India,535,"Navi Mumbai, India",19.30,4.70,12.39,19.21,12.55,138.06


In [24]:
pop_df

,geonameid,name,asciiname,alternatenames,latitude,longitude,feature class,feature code,country code,cc2,admin1 code,admin2 code,admin3 code,admin4 code,population,elevation,dem,timezone,modification date
0,3040051,les Escaldes,LES ESCALDES,"Ehskal'des-Ehndzhordani,Escaldes,Escaldes-Engo...",42.50729,1.53414,P,PPLA,AD,NaN,8,NaN,NaN,NaN,15853,NaN,1033.0,Europe/Andorra,10/15/08
1,3041563,Andorra la Vella,ANDORRA LA VELLA,"ALV,Ando-la-Vyey,Andora,Andora la Vela,Andora ...",42.50779,1.52109,P,PPLC,AD,NaN,7,NaN,NaN,NaN,20430,NaN,1037.0,Europe/Andorra,5/30/10
2,290594,Umm al Qaywayn,UMM AL QAYWAYN,"Oumm al Qaiwain,Oumm al QaÌøwaÌøn,Um al Kawain...",25.56473,55.55517,P,PPLA,AE,NaN,7,NaN,NaN,NaN,44411,NaN,2.0,Asia/Dubai,10/7/14
3,291074,Ras al-Khaimah,RAS AL-KHAIMAH,"Julfa,Khaimah,RKT,Ra's al Khaymah,Ra's al-Chai...",25.78953,55.9432,P,PPLA,AE,NaN,5,NaN,NaN,NaN,115949,NaN,2.0,Asia/Dubai,12/5/15
4,291696,Khawr Fakkn,KHAWR FAKKAN,"Fakkan,Fakkn,Khawr Fakkan,Khawr Fakkn,Khaw...",25.33132,56.34199,P,PPL,AE,NaN,6,NaN,NaN,NaN,33575,NaN,20.0,Asia/Dubai,10/25/13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23464,894701,Bulawayo,BULAWAYO,"BUQ,Bulavajas,Bulavajo,Bulavejo,Bulawayo,Gorad...",-20.15,28.58333,P,PPLA,ZW,NaN,9,NaN,NaN,NaN,699385,NaN,1348.0,Africa/Harare,8/3/10
23465,895061,Bindura,BINDURA,"Bindura,Bindura Town,Kimberley Reefs,_Ô_ü___«...",-17.30192,31.33056,P,PPLA,ZW,NaN,3,NaN,NaN,NaN,37423,NaN,1118.0,Africa/Harare,8/3/10
23466,895269,Beitbridge,BEITBRIDGE,"Bajtbridz,Bajtbridzh,Beitbridge,Beitbridzas,Be...",-22.21667,30,P,PPL,ZW,NaN,7,NaN,NaN,NaN,26459,NaN,461.0,Africa/Harare,3/12/13
23467,1085510,Epworth,EPWORTH,Epworth,-17.89,31.1475,P,PPLX,ZW,NaN,10,NaN,NaN,NaN,123250,NaN,1508.0,Africa/Harare,1/19/12
